# 개요

# Prepare

# 1. 초기 세팅

## Library & data load

In [574]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [885]:
# 2020, 2021 데이터 합치기
# 데이터 불러오기
df_lst = []
for i in range(1,11):
    if i < 13:
        if i < 10:
            name = "auction_20220"+str(i)
            df_name = "auction_20220"+str(i)
        else:
            name = "auction_2022"+str(i)
            df_name = "auction_2022"+str(i)
    name = pd.read_csv("/Users/luci031/Desktop/Coding/g_auction/data_origin/"+df_name+".csv",encoding='euc_kr')
    df_lst.append(name)

    # 데이터 전처리
for df in df_lst:
    df.reset_index(inplace=True,drop=True)
    if 'Unnamed: 0' in df.columns:
        df.drop(columns=['Unnamed: 0'],inplace=True)

# 모든 데이터 통합
df = df_lst[0]
for dfs in df_lst[1:]:
    df = pd.concat([df,dfs])
df.reset_index(inplace=True,drop=True)

In [886]:
# 처리용 df와 오리지널 버전 남겨놓음
# names = '2017'
# address = "/Users/luci031/Desktop/Coding/g_auction/data_origin/auction_"+names+".csv"
# df = pd.read_csv(address,encoding='euc_kr',sep=';')
# org_df = pd.read_csv(address,encoding='euc_kr',sep=';')

# Pre-Processing

In [887]:
df

,품목명,단위,등급,가격,산지,친환경구분,입력일
0,[감귤]감귤(기타),5kg,4등,5000,제주,일반,20220103
1,[감귤]감귤(기타),5kg,6등,4500,제주,일반,20220103
2,[감귤]감귤(기타),5kg,상(2등),7000,제주특별자치도 서귀,일반,20220103
3,[감귤]감귤(기타),5kg,특(1등),5000,제주 서귀포시,일반,20220103
4,[감귤]감귤(기타),5kg,6등,4500,제주,일반,20220103
...,...,...,...,...,...,...,...
9644183,[홍합국산]홍합국산,10kg,보통(3등),14750,전남 여수\t,일반,20221031
9644184,[홍합국산]홍합국산,10kg,보통(3등),15000,"경남 마산(고성,진해)",일반,20221031
9644185,[홍합국산]홍합국산,10kg,보통(3등),27000,"경남 마산(고성,진해)",일반,20221031
9644186,[홍합국산]홍합국산,10kg,보통(3등),14750,전남 여수\t,일반,20221031


In [888]:
# 결측치 확인
df.isnull().sum()

품목명          0
단위           0
등급           0
가격           0
산지       14240
친환경구분        0
입력일          0
dtype: int64

- 결측치 처리

In [889]:
# 결측치 제거
df = df.dropna(axis=0)

- 영문명으로 전환

In [890]:
df = df.rename(columns = {'품목명':'prd',
                          '단위':'scale','등급':'class',
                          '가격':'price',
                          '산지':'origin',
                          '친환경구분':'eco',
                          '입력일':'reg_date'})

## column 별 정보

### class

In [891]:
# 등급명 변환
df111 = pd.DataFrame([['특(1등)', 1], ['상(2등)',2], ['보통(3등)',3], ['9등(등외)',9], ['없음',9], ['5등',5], ['4등',4], ['6등',6], ['7등',7],['8등',8]],columns=['class','new_class'])
df2 = df.merge(df111)
df2 = df2.drop(columns='class')

In [892]:
df2['new_class'].value_counts()

1    7769805
2     975235
9     423646
3     378178
5      38244
4      32669
6       8439
7       2749
8        983
Name: new_class, dtype: int64

### eco

In [893]:
# 영문명으로 전환
# 전환기 -> 유기농 재배로 전환중
df2.loc[df2["eco"]=="일반","eco"] = "normal"
df2.loc[df2["eco"]=="우수농산물","eco"] = "good_prd"
df2.loc[df2["eco"]=="무농약","eco"] = "ecofriendly"
df2.loc[df2["eco"]=="유기농","eco"] = "organic"
df2.loc[df2["eco"]=="품질인증","eco"] = "certified"
df2.loc[df2["eco"]=="전환기","eco"] = "ing"
df2.loc[df2["eco"]=="저농약","eco"] = "low"
df2.loc[df2["eco"]=="산지안전성","eco"] = "safe"

In [894]:
df2['eco'].value_counts()

normal         8216554
good_prd        896677
ecofriendly     396561
certified        85824
ing              17004
organic           8835
low               8493
Name: eco, dtype: int64

### reg_date

In [895]:
df2['reg_date'] = df2['reg_date'].astype('str')
df2['reg_date'] = pd.to_datetime(df2['reg_date'].astype('str'),format='%Y%m%d')

### price

In [896]:
# 계산을 위해 scale 값 미리 변경
# 현재 값들은 string이므로 float 형태로 변환할 필요가 있음
# 몇몇 이상치 처리도 진행
def scale_checker(scale):
  if scale.startswith('.')==True:
    return format(float('0'+scale[:-2]),'.2f')
  else:
    return format(float((scale[:-2])),'.2f')
df2['scale'] = df2['scale'].apply(scale_checker)
df2['scale'] = df2['scale'].astype('float')
# df2['scale'] = format(round(df2['scale'].apply(pd.to_numeric),2),'.2f')

In [897]:
df2['price'] = pd.to_numeric(df2['price'])

In [898]:
df2[df2['price']<0]

,prd,scale,price,origin,eco,reg_date,new_class


In [899]:
# 1원짜리 거래 발견, 너무 거래값이 작은 값들은 삭제
pd.options.display.float_format = '{:.5f}'.format
df2['price'].describe()

count    9629948.00000
mean       22492.11791
std        85709.59148
min            0.00000
25%         9000.00000
50%        15000.00000
75%        26500.00000
max     99999999.00000
Name: price, dtype: float64

In [900]:
df2 = df2.drop(df2[df2['price']<500].index)
df2 = df2.drop(df2[df2['scale']==0].index)

In [901]:
# kg 당 가격 생성
df2['price_kg'] = round((df2['price']/df2['scale']),2)

### prd
- 품목명 정리
- 품목 종류 정리

In [902]:
df2['price_kg'].describe()

count    9623537.00000
mean        4323.13788
std        23577.69666
min            1.00000
25%         1650.00000
50%         2900.00000
75%         5200.00000
max     49999999.50000
Name: price_kg, dtype: float64

In [903]:
# 대부분 [상품 종류]상품명 으로 정리되어 있는걸 알 수 있음
df2['prd'].value_counts()

[오이]백다다기            285265
[참외]참외              232587
[깻잎]깻잎              206628
[호박]애호박             203596
[딸기]설향              188415
                     ...  
[바나나(수입)]바나나(몽키)         1
[참돔국산]참돔국산               1
[대추]선추(대추)               1
[민어국산]민어국산               1
[건고추]건고추(수입)             1
Name: prd, Length: 2849, dtype: int64

In [904]:
# 정리를 위해 상품명 삭제
def prd_sort(words):
  p = re.compile('[ㄱ-힣]+')
  word = re.search(p,words)
  return word.group()

df2['prd'] = df2['prd'].apply(prd_sort)

In [905]:
# 기타 품목 삭제
df2 = df2.drop(df2[df2['prd']=='기타'].index)

In [906]:
# 국산,수입산,깐,기타 삭제
def prd_sort2(words):
  p = re.compile('[^국산수입깐기타]+')
  word = re.search(p,str(words))
  return word.group()

df2["prd"] = df2["prd"].apply(prd_sort2)


In [907]:
# 무우 -> 무로 통합
df2.loc[df2["prd"]=="무우","prd"] = "무"
#각굴 -> 굴로 통합
df2.loc[df2["prd"]=="각굴","prd"] = "굴"
df2.loc[df2["prd"]=="봉지굴","prd"] = "굴"

In [908]:
# 김장 품목 선택
df2 = df2[(df2['prd']=='굴') | 
    (df2['prd']=='생강') | 
    (df2['prd']=='마늘') | 
    (df2['prd']=='양파') | 
    (df2['prd']=='사과') | 
    (df2['prd']=='배') | 
    (df2['prd']=='배추') | 
    (df2['prd']=='총각무') | 
    (df2['prd']=='쪽파') | 
    (df2['prd']=='갓') | 
    (df2['prd']=='미나리')]

In [909]:
df2['prd'].value_counts()

사과     204315
배추     189478
양파     123323
배       82337
미나리     54308
마늘      45827
쪽파      40456
굴       14345
갓        8061
생강       6708
총각무      1394
Name: prd, dtype: int64

In [910]:
df3 = df2

### scale

In [911]:
np.set_printoptions(precision=2, suppress=True)
df3['scale'].unique()

array([  20.  ,   15.  ,   10.  ,    8.  ,   12.  ,    5.  ,    3.  ,
          4.  ,    2.  ,    6.  ,    2.5 ,    0.2 ,    1.  ,    7.5 ,
         18.  ,    1.6 ,  107.1 ,   11.  ,   17.  ,   14.  ,    2.6 ,
         13.  ,    2.2 ,    4.5 ,    6.5 ,    4.2 ,    5.8 ,    9.  ,
          4.7 ,    8.1 ,    8.2 ,    0.5 ,    3.2 ,    4.8 ,   15.01,
        142.  ,   19.  ,    2.7 ,    0.3 ,    1.2 ,    1.5 ,   25.  ,
         30.  ,    0.25,    1.8 ,    1.3 ,   32.  ,   26.  ,    0.7 ,
         35.  ,    0.4 ,   28.  ,   21.  ,    3.5 ,   23.  ,    7.  ,
         27.  ,   16.  ,   24.  ,   51.  ,    0.9 ,    3.7 ,    0.8 ,
          0.6 ,    5.5 ,    3.6 ,   10.01,   22.  ,    3.8 ,   15.03,
         15.02,   29.  ,    2.8 ,    7.4 ,    6.2 , 1000.  ,    5.3 ,
          5.2 ,    1.1 ,    2.3 ,    6.1 ,    9.2 ,    1.4 ,    1.32,
          6.4 ,    2.4 ,    9.6 ,    8.3 ,    0.1 ,    8.5 ,    9.5 ,
          7.6 ,    0.46,  155.  ,    4.1 ,   63.  ,    5.6 ])

In [912]:
df3[df3['scale']>=50]

,prd,scale,price,origin,eco,reg_date,new_class,price_kg
78984,배,107.10000,37000,경기 안성시,normal,2022-01-19,2,345.47000
990129,배추,142.00000,3600,경기도 평택시,normal,2022-10-22,2,25.35000
990130,배추,142.00000,3060,경기도 평택시,normal,2022-10-22,2,21.55000
1377973,사과,51.00000,24000,충남 서산시,normal,2022-01-18,1,470.59000
1377981,사과,51.00000,17000,충남 서산시,normal,2022-01-18,1,333.33000
1377982,사과,51.00000,24000,충남 서산시,normal,2022-01-18,1,470.59000
1377983,사과,51.00000,17000,충남 서산시,normal,2022-01-18,1,333.33000
5160035,양파,1000.00000,800000,경북 김천시,normal,2022-06-18,1,800.00000
5403841,양파,1000.00000,800000,경북 김천시,normal,2022-06-27,1,800.00000
8560362,배추,142.00000,6000,경기도 평택시,normal,2022-10-22,1,42.25000


In [913]:
# 50kg 이상 거래된 건들 이상치라 판단하고 확인 진행
df3[df3['scale']>=50]['prd'].unique()

array(['배', '배추', '사과', '양파'], dtype=object)

In [856]:
df3[df3['prd']=='배추']['price_kg'].describe()

count   137685.00000
mean      1304.96790
std       2785.24340
min         41.67000
25%        650.00000
50%       1112.50000
75%       1650.00000
max     625675.00000
Name: price_kg, dtype: float64

In [914]:
# 스케일 오표기된 항목 수정
# 양파,호박 정상 거래
# 이상치 항목 삭제
df3 = df3.drop(df3[(df3['scale']>=50)&(df3['prd']!='양파')].index)
# df3[(df3['prd']=='바나나')&(df3['scale']>=10)]


In [915]:
df3[df3['scale']>=50]

,prd,scale,price,origin,eco,reg_date,new_class,price_kg
5160035,양파,1000.00000,800000,경북 김천시,normal,2022-06-18,1,800.00000
5403841,양파,1000.00000,800000,경북 김천시,normal,2022-06-27,1,800.00000


### origin

In [916]:
# 도,시, 국산, 수입 표기 분류 필요
df3['origin'].unique()

array(['전남 무안군', '전북 임실군', '전남 영암군', '경북', '전남 해남군', '전남 함평군', '전남 신안군',
       '충남 태안군', '전남 진도군', '전남 나주시', '제주', '제주 서귀포시(2)', '경기 안성시',
       '전남 고흥군', '충남 당진시', '강원 평창군', '경기 이천시', '충남 예산군', '강원 홍천군',
       '전북 완주군', '경남 함양군', '강원 인제군', '충남 홍성군', '충남 서산시', '경기 화성시',
       '전북 김제시', '전북 부안군', '강원 정선군', '경남 창녕군', '대구 남구', '강원 태백시',
       '충남 천안시', '경북 구미시', '경북 경산시', '경북 영양군', '경북 영천시', '경남 합천군',
       '경기 여주시 가남읍', '경기 이천시 장호원읍', '경기 평택시', '경기 안성시 서운면', '경기 여주군',
       '충남 아산시', '경기도 남양주시', '충남', '경기도 평택시', '경기 여주시', '충북 진천군',
       '경북 영덕군', '광주 서구', '충북 음성군', '충북 보은군', '경북 청송군', '경북 봉화군',
       '경북 의성군', '대구 달성군', '경북 영주시', '경북 김천시', '전남/고흥군', '충남 부여군',
       '경기 남양주시', '충남 연기군', '충북 충주시', '경북 안동시', '경남 거창군', '전북', '경북 고령군',
       '충북 영동군', '충북', '경기', '경기 평택시 현덕면', '전북 무주군', '전북 고창군', '경기도 안성시',
       '경기 김포시', '경북 상주시', '경기 구리시', '전남 완도군', '전남 장흥군', '충북 단양군',
       '전북 장수군', '제주 북제주군', '충북 제천시', '경기 평택시 안중읍', '경북 문경시 가은읍',
       '경기도 김포시', '전남', '제주 서귀포시', '강원 횡성군'

In [917]:
# 해당 조건에 맞는 값 추출

splits = df3['origin'].str.split(' ')
cacul = splits.apply(lambda x: pd.Series(x))
df66 = pd.merge(df3,cacul, left_index=True, right_index=True, how='inner')
df66

,prd,scale,price,origin,eco,reg_date,new_class,price_kg,0,1,2
102,마늘,20.00000,154650,전남 무안군,normal,2022-01-03,4,7732.50000,전남,무안군,NaN
103,마늘,20.00000,154650,전남 무안군,normal,2022-01-03,4,7732.50000,전남,무안군,NaN
104,마늘,20.00000,154650,전남 무안군,normal,2022-01-03,4,7732.50000,전남,무안군,NaN
105,마늘,20.00000,154650,전남 무안군,normal,2022-01-03,4,7732.50000,전남,무안군,NaN
107,마늘,20.00000,154650,전남 무안군,normal,2022-01-03,4,7732.50000,전남,무안군,NaN
...,...,...,...,...,...,...,...,...,...,...,...
9629932,양파,15.00000,11000,전남 무안군,normal,2022-10-27,8,733.33000,전남,무안군,NaN
9629933,양파,15.00000,11000,전남 무안군,normal,2022-10-27,8,733.33000,전남,무안군,NaN
9629936,양파,15.00000,17000,전남 무안군,normal,2022-10-28,8,1133.33000,전남,무안군,NaN
9629937,양파,15.00000,17000,전남 무안군,normal,2022-10-28,8,1133.33000,전남,무안군,NaN


In [918]:
# 세부 카테고리 정리 (state, city 편집 시 여기서 시작)

df4 = df66.rename(columns = {0:'state',1:'city'})

# 2,3은 필요없는 정보이므로 삭제
df4 = df4.drop(columns=2)

In [919]:
df4.head(1)

,prd,scale,price,origin,eco,reg_date,new_class,price_kg,state,city
102,마늘,20.00000,154650,전남 무안군,normal,2022-01-03,4,7732.50000,전남,무안군


In [920]:
df4['state'].unique()

array(['전남', '전북', '경북', '충남', '제주', '경기', '강원', '경남', '대구', '경기도', '충북',
       '광주', '전남/고흥군', '세종', '서울', '대구시', '제주도', '인천', '강원도', '충북청주시상당구',
       '청주시', '충청남도', '서울시', '세종특별자치시', '전라북도', '경남/남해군', '경상북도', '부산',
       '울산', '전주시', '부산시', '중국', '수입산', '경북/문경시', '광주시', '수원시', '가락동',
       '제주시', '성남시', '경남/합천군', '부천시', '서귀포시', '제주/서귀포', '전라남도', '인천시',
       '충남/아산시', '광주광역시', '서울특별시', '대전', '충남/홍성군', '국내산\t', '수입', '이시원',
       '경상남도', '부산광역시', '충청북도'], dtype=object)

In [921]:
# 시,도 제거
def name_sort(words):
  p = re.compile('[^시도]+')
  word = re.search(p,str(words))
  return word.group()

df4["state"] = df4["state"].apply(name_sort)

# 오타도 제거

def name_sort2(words):
  p = re.compile('[^\t]+')
  word = re.search(p,words)
  return word.group()

df4["state"] = df4["state"].apply(name_sort2)

In [922]:
df4["state"].unique()

array(['전남', '전북', '경북', '충남', '제주', '경기', '강원', '경남', '대구', '충북', '광주',
       '전남/고흥군', '세종', '서울', '인천', '충북청주', '청주', '충청남', '세종특별자치', '전라북',
       '경남/남해군', '경상북', '부산', '울산', '전주', '중국', '수입산', '경북/문경', '수원',
       '가락동', '성남', '경남/합천군', '부천', '서귀포', '제주/서귀포', '전라남', '충남/아산',
       '광주광역', '서울특별', '대전', '충남/홍성군', '국내산', '수입', '이', '경상남', '부산광역',
       '충청북'], dtype=object)

In [923]:
df4[df4['state']=='공통출하처']

,prd,scale,price,origin,eco,reg_date,new_class,price_kg,state,city


In [924]:
# state
# 특이케이스 제거
df4 = df4.drop(df4[(df4['state']>='없음')].index)
df4 = df4.drop(df4[(df4['state']>='해당사항없')].index)

# 국내
df4.loc[df4["state"]=="국내산","city"] = "미상"
df4.loc[df4["state"]=="국내산","state"] = "국산"
df4.loc[df4["state"]=="원양산","city"] = "미상"
df4.loc[df4["state"]=="공통출하처","city"] = "미상"
df4.loc[df4["state"]=="공통출하처","state"] = "국산"

## 도단위
df4.loc[df4["state"]=="충청북","state"] = "충북"
df4.loc[df4["state"]=="충청남","state"] = "충남"
df4.loc[df4["state"]=="전라북","state"] = "전북"
df4.loc[df4["state"]=="전라남","state"] = "전남"
df4.loc[df4["state"]=="경상북","state"] = "경북"
df4.loc[df4["state"]=="경상남","state"] = "경남"

## 시단위
### 광역시

df4.loc[df4["state"]=="대전","city"] = "대전"
df4.loc[df4["state"]=="대구","city"] = "대구"
df4.loc[df4["state"]=="서울특별","city"] = "서울"
df4.loc[df4["state"]=="서울특별","state"] = "서울"
df4.loc[df4["state"]=="인천광역","city"] = "인천"
df4.loc[df4["state"]=="인천광역","state"] = "인천"
df4.loc[df4["state"]=="부산","city"] = "부산"
df4.loc[df4["state"]=="부산광역","city"] = "부산"
df4.loc[df4["state"]=="부산광역","state"] = "부산"
df4.loc[df4["state"]=="가락동","city"] = "서울"
df4.loc[df4["state"]=="가락동","state"] = "서울"
df4.loc[df4["state"]=="세종","city"] = "세종"
df4.loc[df4["state"]=="세종특별자치","city"] = "세종"
df4.loc[df4["state"]=="세종특별자치","state"] = "세종"
df4.loc[df4["state"]=="대전","city"] = "대전"
df4.loc[df4["state"]=="제주","city"] = "제주"
df4.loc[df4["state"]=="제주특별자치","city"] = "제주"
df4.loc[df4["state"]=="제주특별자치","state"] = "제주"
df4.loc[df4["state"]=="제주특별자","city"] = "제주"
df4.loc[df4["state"]=="제주특별자","state"] = "제주"
df4.loc[df4["state"]=="제주/서귀포","city"] = "제주"
df4.loc[df4["state"]=="제주/서귀포","state"] = "제주"
df4.loc[df4["state"]=="서귀포","city"] = "제주"
df4.loc[df4["state"]=="서귀포","state"] = "제주"
df4.loc[df4["state"]=="세종","city"] = "세종"
df4.loc[df4["state"]=="울산","city"] = "울산"
df4.loc[df4["state"]=="울산","state"] = "울산"
df4.loc[df4["state"]=="광주광역","state"] = "광주"
df4.loc[df4["state"]=="광주광역","city"] = "광주"

### 일반시
df4.loc[df4["state"]=="성남","city"] = "성남"
df4.loc[df4["state"]=="성남","state"] = "경기"
df4.loc[df4["state"]=="경기성남","city"] = "성남"
df4.loc[df4["state"]=="경기성남","state"] = "경기"
df4.loc[df4["state"]=="전주","city"] = "전주"
df4.loc[df4["state"]=="전주","state"] = "전북"
df4.loc[df4["state"]=="청주","city"] = "청주"
df4.loc[df4["state"]=="청주","state"] = "충북"
df4.loc[df4["state"]=="수원","city"] = "수원"
df4.loc[df4["state"]=="수원","state"] = "경기"
df4.loc[df4["state"]=="부천","city"] = "부천"
df4.loc[df4["state"]=="부천","state"] = "경기"
df4.loc[df4["state"]=="경기고양","city"] = "고양"
df4.loc[df4["state"]=="경기고양","state"] = "경기"
df4.loc[df4["state"]=="충북청주","city"] = "청주"
df4.loc[df4["state"]=="충북청주","state"] = "충북"
df4.loc[df4["state"]=="충북청주시흥덕구","city"] = "청주"
df4.loc[df4["state"]=="충북청주시흥덕구","state"] = "충북"
df4.loc[df4["state"]=="전북전주","city"] = "전주"
df4.loc[df4["state"]=="전북전주","state"] = "전북"
df4.loc[df4["state"]=="경북/문경","city"] = "문경"
df4.loc[df4["state"]=="경북/문경","state"] = "경북"
df4.loc[df4["state"]=="포항","city"] = "포항"
df4.loc[df4["state"]=="포항","state"] = "경북"
df4.loc[df4["state"]=="경북/안동","city"] = "안동"
df4.loc[df4["state"]=="경북/안동","state"] = "경북"
df4.loc[df4["state"]=="전남/무안군","city"] = "전남"
df4.loc[df4["state"]=="전남/무안군","state"] = "무안"
df4.loc[df4["state"]=="경북/구미","city"] = "구미"
df4.loc[df4["state"]=="경북/구미","state"] = "경북"
df4.loc[df4["state"]=="경북/상주","city"] = "상주"
df4.loc[df4["state"]=="경북/상주","state"] = "경북"
df4.loc[df4["state"]=="경북/영천","city"] = "영천"
df4.loc[df4["state"]=="경북/영천","state"] = "경북"
df4.loc[df4["state"]=="경남/합천군","city"] = "합천"
df4.loc[df4["state"]=="경남/합천군","state"] = "경남"
df4.loc[df4["state"]=="전남/고흥군","city"] = "고흥"
df4.loc[df4["state"]=="전남/고흥군","state"] = "전남"
df4.loc[df4["state"]=="경남/남해군","state"] = "경남"
df4.loc[df4["state"]=="경남/남해군","city"] = "남해"
df4.loc[df4["state"]=="무안","state"] = "전남"
df4.loc[df4["state"]=="서해안","city"] = "미상"
df4.loc[df4["state"]=="서해안","state"] = "국산"
df4.loc[df4["state"]=="남해안","city"] = "미상"
df4.loc[df4["state"]=="남해안","state"] = "국산"


# 외국
## 아시아
df4.loc[df4["state"]=="칠레","city"] = "칠레"
df4.loc[df4["state"]=="칠레","state"] = "수입"
df4.loc[df4["state"]=="일본","city"] = "일본"
df4.loc[df4["state"]=="일본","state"] = "수입"
df4.loc[df4["state"]=="베트남","city"] = "베트남"
df4.loc[df4["state"]=="베트남","state"] = "수입"
df4.loc[df4["state"]=="우즈베크","city"] = "우즈베키스탄"
df4.loc[df4["state"]=="우즈베크","state"] = "수입"
df4.loc[df4["state"]=="이스라엘","city"] = "이스라엘"
df4.loc[df4["state"]=="이스라엘","state"] = "수입"
df4.loc[df4["state"]=="터키","city"] = "터키"
df4.loc[df4["state"]=="터키","state"] = "수입"
df4.loc[df4["state"]=="중국","city"] = "중국"
df4.loc[df4["state"]=="중국","state"] = "수입"
df4.loc[df4["state"]=="중국(CN)","city"] = "중국"
df4.loc[df4["state"]=="중국(CN)","state"] = "수입"
df4.loc[df4["state"]=="타이","city"] = "태국"
df4.loc[df4["state"]=="타이","state"] = "수입"
df4.loc[df4["state"]=="태국","city"] = "태국"
df4.loc[df4["state"]=="태국","state"] = "수입"
df4.loc[df4["state"]=="필리핀","city"] = "필리핀"
df4.loc[df4["state"]=="필리핀","state"] = "수입"
df4.loc[df4["state"]=="우즈베키스탄","city"] = "우즈베키스탄"
df4.loc[df4["state"]=="우즈베키스탄","state"] = "수입"
df4.loc[df4["state"]=="대만","city"] = "대만"
df4.loc[df4["state"]=="대만","state"] = "수입"
df4.loc[df4["state"]=="베트남","city"] = "베트남"
df4.loc[df4["state"]=="베트남","state"] = "수입"
df4.loc[df4["state"]=="이란","city"] = "이란"
df4.loc[df4["state"]=="이란","state"] = "수입"
df4.loc[df4["state"]=="러","city"] = "러시아"
df4.loc[df4["state"]=="러","state"] = "수입"
df4.loc[df4["state"]=="캄보디아","city"] = "캄보디아"
df4.loc[df4["state"]=="캄보디아","state"] = "수입"
df4.loc[(df4["state"]=="인")&(df4["origin"]=="인도네시아"),"city"] = "인도네시아"
df4.loc[(df4["state"]=="인")&(df4["origin"]=="인도네시아"),"state"] = "수입"
df4.loc[(df4["state"]=="인")&(df4["origin"]=="인도"),"city"] = "인도"
df4.loc[(df4["state"]=="인")&(df4["origin"]=="인도"),"state"] = "수입"

## 아메리카

df4.loc[df4["state"]=="캘리포니아","city"] = "미국"
df4.loc[df4["state"]=="캘리포니아","state"] = "수입"
df4.loc[df4["state"]=="포클랜드","city"] = "영국"
df4.loc[df4["state"]=="포클랜드","state"] = "수입"
df4.loc[df4["state"]=="콜롬비아","city"] = "콜롬비아"
df4.loc[df4["state"]=="콜롬비아","state"] = "수입"
df4.loc[df4["state"]=="에콰","city"] = "에콰도르"
df4.loc[df4["state"]=="에콰","state"] = "수입"
df4.loc[df4["state"]=="에쿠아","city"] = "에콰도르"
df4.loc[df4["state"]=="에쿠아","state"] = "수입"
df4.loc[df4["state"]=="에콰돌","city"] = "에콰도르"
df4.loc[df4["state"]=="에콰돌","state"] = "수입"
df4.loc[df4["state"]=="과테말라","city"] = "과테말라"
df4.loc[df4["state"]=="과테말라","state"] = "수입"
df4.loc[df4["state"]=="코스타리카","city"] = "코스타리카"
df4.loc[df4["state"]=="코스타리카","state"] = "수입"
df4.loc[df4["state"]=="멕","city"] = "멕시코"
df4.loc[df4["state"]=="멕","state"] = "수입"
df4.loc[df4["state"]=="페루","city"] = "페루"
df4.loc[df4["state"]=="페루","state"] = "수입"
df4.loc[df4["state"]=="미국","city"] = "미국"
df4.loc[df4["state"]=="미국","state"] = "수입"
df4.loc[df4["state"]=="아르헨티나","city"] = "아르헨티나"
df4.loc[df4["state"]=="아르헨티나","state"] = "수입"
df4.loc[df4["state"]=="브라질","city"] = "브라질"
df4.loc[df4["state"]=="브라질","state"] = "수입"

## 유럽
df4.loc[df4["state"]=="스페인","city"] = "스페인"
df4.loc[df4["state"]=="스페인","state"] = "수입"
df4.loc[df4["state"]=="우크라이나","city"] = "우크라이나"
df4.loc[df4["state"]=="우크라이나","state"] = "수입"
df4.loc[df4["state"]=="이탈리아","city"] = "이탈리아"
df4.loc[df4["state"]=="이탈리아","state"] = "수입"
df4.loc[df4["state"]=="불가리아","city"] = "불가리아"
df4.loc[df4["state"]=="불가리아","state"] = "수입"
df4.loc[df4["state"]=="그리스","city"] = "그리스"
df4.loc[df4["state"]=="그리스","state"] = "수입"

## 아프리카
df4.loc[df4["state"]=="남아프리카","city"] = "남아공"
df4.loc[df4["state"]=="남아프리카","state"] = "수입"
df4.loc[df4["state"]=="남아프리카공화국","city"] = "남아공"
df4.loc[df4["state"]=="남아프리카공화국","state"] = "수입"
df4.loc[df4["state"]=="수입남아프리카공화국","city"] = "남아공"
df4.loc[df4["state"]=="수입남아프리카공화국","state"] = "수입"


## 오세아니아
df4.loc[df4["state"]=="뉴질랜드","city"] = "뉴질랜드"
df4.loc[df4["state"]=="뉴질랜드","state"] = "수입"
df4.loc[df4["state"]=="오스트레일","city"] = "호주"
df4.loc[df4["state"]=="오스트레일","state"] = "수입"
df4.loc[df4["state"]=="오스트레일리아","city"] = "호주"
df4.loc[df4["state"]=="오스트레일리아","state"] = "수입"
df4.loc[df4["state"]=="호주","city"] = "호주"
df4.loc[df4["state"]=="호주","state"] = "수입"
df4.loc[df4["state"]=="통가","city"] = "통가"
df4.loc[df4["state"]=="통가","state"] = "수입"
df4.loc[df4["state"]=="뉴칼레","city"] = "뉴칼레도니아"
df4.loc[df4["state"]=="뉴칼레","state"] = "수입"
df4.loc[df4["state"]=="뉴","city"] = "뉴칼레도니아"
df4.loc[df4["state"]=="뉴","state"] = "수입"
df4.loc[df4["state"]=="누벨칼레","city"] = "뉴칼레도니아"
df4.loc[df4["state"]=="누벨칼레","state"] = "수입"
# 기타
df4.loc[df4["state"]=="수입산","state"] = "수입"
df4.loc[df4["state"]=="기타외국","city"] = "미상"
df4.loc[df4["state"]=="기타외국","state"] = "수입"
df4.drop(df4[df4["state"]=="미등록"].index,inplace=True)

In [925]:
df4['state'].unique()

array(['경북', '경기', '강원', '경남', '대구', '광주', '세종', '서울', '부산', '수입', '제주',
       '대전', '국산'], dtype=object)

In [926]:
# city
# 우선 결측치 제거
df4.loc[df4["city"].isnull(),"city"] = "미상"

In [927]:
# 이후 시,군 등 제거
def name_sort(words):
  p = re.compile('[^시군]+')
  word = re.search(p,words)
  return word.group()

df4["city"] = df4["city"].apply(name_sort)

In [928]:
# \t 도 제거
def name_sort2(words):
  p = re.compile('[^\t]+')
  word = re.search(p,words)
  return word.group()

df4["city"] = df4["city"].apply(name_sort2)

In [929]:
df4['city'].unique()

array(['미상', '안성', '평창', '이천', '홍천', '함양', '인제', '화성', '정선', '창녕', '대구',
       '태백', '구미', '경산', '영양', '영천', '합천', '여주', '평택', '남양주', '영덕', '서구',
       '청송', '봉화', '의성', '영주', '김천', '안동', '거창', '고령', '김포', '상주', '구리',
       '문경', '횡성', '세종', '남구', '파주', '송파구', '양구', '포천', '청도', '산청', '밀양',
       '양평', '강동구', '광산구', '용인', '예천', '의령', '위', '성남', '오산', '하남', '철원',
       '고양', '광주', '함안', '울진', '남해', '칠곡', '영월', '연천', '삼척', '강서구', '화천',
       '춘천', '성주', '양양', '마포구', '중랑구', '강릉', '고성', '안양', '가평', '양주', '하동',
       '도봉구', '양산', '부산', '포항', '창원', '은평구', '중국', '진주', '경주', '중구', '수원',
       '서울', '김해', '안산', '흥', '부천', '제주', '사천', '광명', '의정부', '동대문구',
       '구로구', '북구', '원주', '용산구', '양천구', '영등포구', '영', '대전', '통영',
       '마산(고성,진해)', '삼천포', '안', '거', '합'], dtype=object)

In [930]:
# 나머지 케이스 정리

# 국내
df4.loc[df4["city"]=="기타","city"] = "미상"
## 일반시
df4.loc[df4["city"]=="위","city"] = "군위"
df4.loc[df4["city"]=="포","city"] = "군포"
df4.loc[df4["city"]=="고흥(나로도)","city"] = "고흥"
df4.loc[df4["city"]=="산","city"] = "군산"
df4.loc[df4["city"]=="장항(서천)","city"] = "서천"
df4.loc[df4["city"]=="삼천포","city"] = "사천"
df4.loc[df4["city"]=="삼천포","city"] = "사천"
df4.loc[df4["city"]=="순천(광양)","city"] = "순천"
df4.loc[df4["city"]=="대천(보령)","city"] = "보령"
df4.loc[df4["city"]=="대천(보령)","city"] = "보령"
df4.loc[df4["city"]=="화성(평택)","city"] = "화성"
df4.loc[df4["city"]=="흥","city"] = "시흥"
df4.loc[df4["city"]=="목포(무안)","city"] = "목포"
df4.loc[df4["city"]=="목포(무안)","state"] = "전남"
df4.loc[df4["city"]=="보성(벌교)","city"] = "보성"
df4.loc[df4["city"]=="울진(후포)","city"] = "울진"
df4.loc[df4["city"]=="논","city"] = "논산"
df4.loc[df4["city"]=="논","city"] = "논산"
df4.loc[df4["city"]=="순","city"] = "순창"
df4.loc[df4["city"]=="익","city"] = "익산"
df4.loc[df4["city"]=="미금","city"] = "성남"
df4.loc[df4["city"]=="주문진","city"] = "강릉"
df4.loc[df4["city"]=="울릉도","city"] = "울릉"
df4.loc[df4["city"]=="거제(장승포)","city"] = "거제"
df4.loc[df4["city"]=="남양","city"] = "남양주"
df4.loc[(df4["city"]=="완")&(df4["state"]=="전북"),"city"] = "완주"
df4.loc[(df4["state"]=="전북")&(df4["city"]=="남"),"city"] = "남원"
df4.loc[(df4['prd']=="호박") & (df4['state']=='전북'),"city"] = "전주"
df4.loc[(df4["state"]=="전남")&(df4["city"]=="신"),"city"] = "신안"
df4.loc[(df4["state"]=="전북")&(df4["city"]=="장"),"city"] = "장수"
df4.loc[(df4["state"]=="전남")&(df4["city"]=="진"),"city"] = "진도"
df4.loc[(df4["state"]=="충남")&(df4["city"]=="부"),"city"] = "부여"
df4.loc[(df4["state"]=="경남")&(df4["city"]=="의"),"city"] = "의령"
df4.loc[(df4["state"]=="전남")&(df4["city"]=="함"),"city"] = "함평"
df4.loc[(df4["state"]=="경북")&(df4["city"]=="영"),"city"] = "영주"
df4.loc[(df4["state"]=="경북")&(df4["city"]=="안"),"city"] = "안동"
df4.loc[(df4["state"]=="경남")&(df4["city"]=="거"),"city"] = "거창"
df4.loc[(df4["state"]=="경남")&(df4["city"]=="합"),"city"] = "합천"
# 전라남도 장 -> 장흥, 장성 두개 있으므로 구분 불가, 삭제
df4.drop(df4[(df4["city"]=="장")&(df4["state"]=="전남")].index,inplace=True)

## 광역시
df4.loc[df4["city"]=="강서구","city"] = "서울"
df4.loc[df4["city"]=="은평구","city"] = "서울"
df4.loc[df4["city"]=="강동구","city"] = "서울"
df4.loc[df4["city"]=="마포구","city"] = "서울"
df4.loc[df4["city"]=="중구","city"] = "서울"
df4.loc[df4["city"]=="송파구","city"] = "서울"
df4.loc[df4["city"]=="도봉구","city"] = "서울"
df4.loc[df4["city"]=="중랑구","city"] = "서울"
df4.loc[df4["city"]=="양천구","city"] = "서울"
df4.loc[df4["city"]=="성동구","city"] = "서울"
df4.loc[df4["city"]=="강남구","city"] = "서울"
df4.loc[df4["city"]=="동대문구","city"] = "서울"
df4.loc[df4["city"]=="영등포구","city"] = "서울"
df4.loc[df4["city"]=="영등포","city"] = "서울"
df4.loc[df4["city"]=="서초구","city"] = "서울"
df4.loc[df4["city"]=="구로구","city"] = "서울"
df4.loc[df4["city"]=="동대문","city"] = "서울"
df4.loc[df4["city"]=="금천구","city"] = "서울"
df4.loc[df4["city"]=="동작구","city"] = "서울"
df4.loc[df4["city"]=="용산구","city"] = "서울"
df4.loc[df4["city"]=="광진구","city"] = "서울"
df4.loc[df4["city"]=="종로구","city"] = "서울"
df4.loc[df4["city"]=="서대문구","city"] = "서울"
df4.loc[df4["city"]=="서대문","city"] = "서울"
df4.loc[df4["city"]=="성북구","city"] = "서울"
df4.loc[df4["city"]=="관악구","city"] = "서울"

df4.loc[df4["city"]=="광산구","city"] = "광주"
df4.loc[(df4["state"]=="광주")&(df4["city"]=="남구"),"city"] = "광주"
df4.loc[(df4["state"]=="광주")&(df4["city"]=="북구"),"city"] = "광주"
df4.loc[(df4["state"]=="광주")&(df4["city"]=="서구"),"city"] = "광주"
df4.loc[df4["city"]=="동구","city"] = "광주"
df4.loc[(df4["state"]=="인천")&(df4["city"]=="남구"),"city"] = "인천"
df4.loc[(df4["state"]=="인천")&(df4["city"]=="서구"),"city"] = "인천"
df4.loc[df4["city"]=="미추홀","city"] = "인천"
df4.loc[df4["city"]=="부평구","city"] = "인천"
df4.loc[df4["city"]=="남동구","city"] = "인천"
df4.loc[df4["city"]=="미추홀구","city"] = "인천"
df4.loc[df4["city"]=="계양구","city"] = "인천"
df4.loc[df4["city"]=="마산(고성,진해)","city"] = "창원"
df4.loc[(df4["state"]=="부산")&(df4["city"]=="북구"),"city"] = "부산"
df4.loc[df4["city"]=="오포","city"] = "부산"
df4.loc[df4["city"]=="부산진","city"] = "부산"
df4.loc[df4["city"]=="사상구","city"] = "부산"
df4.loc[df4["city"]=="부산진구","city"] = "부산"
df4.loc[df4["city"]=="북제주","city"] = "제주"
df4.loc[df4["city"]=="남제주","city"] = "제주"
df4.loc[df4["city"]=="고성(거진,대진)","city"] = "고성"

# 해외
## 아시아
df4.loc[df4["city"]=="러","city"] = "러시아"
df4.loc[df4["city"]=="말레이지아","city"] = "말레이시아"
df4.loc[df4["city"]=="타이","city"] = "태국"
df4.loc[df4["city"]=="인도네","city"] = "인도네시아"
df4.loc[df4["city"]=="사우디","city"] = "사우디아라비아"
## 아메리카
df4.loc[df4["city"]=="멕","city"] = "멕시코"
## 아프리카
df4.loc[df4["city"]=="남아공산","city"] = "남아공"
## 유럽
## 오세아니아

In [931]:
#df2[(df2['prd']=='양파') & (df2['price']==23000) & (df2['scale']==15)]['origin'].unique()

In [932]:
df4['city'].unique()

array(['미상', '안성', '평창', '이천', '홍천', '함양', '인제', '화성', '정선', '창녕', '대구',
       '태백', '구미', '경산', '영양', '영천', '합천', '여주', '평택', '남양주', '영덕', '광주',
       '청송', '봉화', '의성', '영주', '김천', '안동', '거창', '고령', '김포', '상주', '구리',
       '문경', '횡성', '세종', '파주', '서울', '양구', '포천', '청도', '산청', '밀양', '양평',
       '용인', '예천', '의령', '군위', '성남', '오산', '하남', '철원', '고양', '함안', '울진',
       '남해', '칠곡', '영월', '연천', '삼척', '화천', '춘천', '성주', '양양', '강릉', '고성',
       '안양', '가평', '양주', '하동', '양산', '부산', '포항', '창원', '중국', '진주', '경주',
       '수원', '김해', '안산', '시흥', '부천', '제주', '사천', '광명', '의정부', '원주', '대전',
       '통영'], dtype=object)

In [933]:
# 기존 city 리스트와 비교
ex_city = pd.read_csv("../cities.csv")
df4[~df4['city'].isin(ex_city['city'])]['city'].unique()

array([], dtype=object)

In [934]:
# 리스트 최신화
new_city = pd.DataFrame(df4['city'].unique(),columns=['city'])
cities = ex_city.append(new_city,ignore_index=True)

/var/folders/nq/s4zx76p53d11lnr0t1dxpxh00000gn/T/ipykernel_6955/1856950528.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cities = ex_city.append(new_city,ignore_index=True)


In [935]:
cities.drop(columns=['Unnamed: 0'],inplace=True)
cities.drop_duplicates(inplace=True)
cities.reset_index(inplace=True,drop=True)

In [936]:
cities.to_csv('../cities.csv')

In [937]:
# 처리 완료하였으니 df4는 백업으로 두고 df5 활용
df5 = df4.drop(columns='origin')

# 전처리 결과

# 파일 저장

In [938]:
# parquet 파일로 변환하여 용량 줄이기
df5.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 417837 entries, 651 to 9626302
Data columns (total 9 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   prd        417837 non-null  object        
 1   scale      417837 non-null  float64       
 2   price      417837 non-null  int64         
 3   eco        417837 non-null  object        
 4   reg_date   417837 non-null  datetime64[ns]
 5   new_class  417837 non-null  int64         
 6   price_kg   417837 non-null  float64       
 7   state      417837 non-null  object        
 8   city       417837 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(2), object(4)
memory usage: 31.9+ MB


In [939]:
df5['new_class']

651        4
652        4
1160       4
1161       4
1162       4
          ..
9624666    9
9625873    9
9625874    9
9626264    7
9626302    7
Name: new_class, Length: 417837, dtype: int64

In [940]:
address2 = "/Users/luci031/Desktop/Coding/g_auction/data_ingredients/auction_"+"202201_10"+".parquet"
df5.to_parquet(address2,engine="pyarrow", compression='gzip')